In [3]:
import pickle
import numpy as np
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
from galaxymodule.galaxy import Galaxy
import tree
import collections
from galaxymodule import galaxy
import load

from analysis.cal_lambda import *


In [4]:
def mk_gal(gal,
            save=False, verbose=False,
            mstar_min=1e9,
            den_lim=1e6, den_lim2=5e6,
            rmin = -1, Rgal_to_reff=5.0, method_com=1, follow_bp=None,
            unit_conversion="code"):
    """
        Refine given catalog (star, DM, gas) to define a realistic galaxy.
        Raw star/DM/gas catalog as give are only rough representation of a galaxy.
        But not much have to be thrown away, either.

        Returns True if a "good" galaxy is made.


        Parameters
        ----------

        Rgal_to_reff:
            Galaxy radius = Reff * Rgal_to_reff.
            By default, Rgal = 5*Reff.
            (E's have smaller R_t_r, L's have larger.)

        Notes
        -----
        1. Since now (as of 2016.03) galaxy calculation is based on the GalaxyMaker results,
        let's just believe and minimize redundant processes such as determining the center of mass,
        system velocity, and checking the presence of (unwanted) substructure.

        2. Assuming all catalog in the code units.

        3. dr, bin size in determining the radial profile cut scales with exponential factor.
        Without scaling, 0.5kpc bin size is too large for ~1kpc galaxies at high-z.

    """
    # Need halo information
    assert (gal.gcat is not None), ("Need a catalog,"
    "use Galaxy.set_halo() and provide x,y,z,vx,vy,vz,r,rvir, at least"
    "Units are.. ?? ")

    # And at least one component.
    star = gal.star
    if star is not None:
        gal.star["x"] = (gal.star["x"] - gal.gcat["x"])*1e3
        gal.star["y"] = (gal.star["y"] - gal.gcat["y"])*1e3
        gal.star["z"] = (gal.star["z"] - gal.gcat["z"])*1e3
        gal.star["m"] *= 1e11#gal.info.msun
    dm = gal.dm
    if dm is not None:
        gal.dm["x"] = (gal.dm["x"] - gal.gcat["x"])*1e3
        gal.dm["y"] = (gal.dm["y"] - gal.gcat["y"])*1e3
        gal.dm["z"] = (gal.dm["z"] - gal.gcat["z"])*1e3
        gal.dm["m"] *= gal.info.msun
    cell = gal.cell

    assert (gal._has_star or gal._has_dm or gal._has_cell), ("At least"
    "one of three(star, dm, gas) component is needed")

    # all tests passed.
    if verbose:
        print("Making a galaxy:", gal.meta.id)
        print("SAVE:", save)
        print("Halo size:", gal.gcat['rvir'])

    #
    member="Reff"

    # galaxy center from GalaxyMaker. - good enough.
    xc = gal.gcat['x']
    yc = gal.gcat['y']
    zc = gal.gcat['z']

    if verbose: print("xc, yc, zc =", xc, yc, zc)

    rgal_tmp = min([gal.gcat['r'] * 1e3, 30]) # gcat["rvir"] in kpc
    if verbose: print("Rgal_tmp", rgal_tmp)
    print("gal.debug",gal.debug)
    dense_enough = radial_profile_cut(gal, star['x'], star['y'], star['m'],
                         star['vx'], star['vy'], star['vz'],
                         den_lim=den_lim, den_lim2=den_lim2,
                         mag_lim=25,
                         nbins=int(rgal_tmp/0.5),
                         dr=0.5 * gal.info.aexp,
                         rmax=rgal_tmp,
                         debug=gal.debug)

    if not dense_enough:
        print("Not dense enough")
        return False
    ind = np.where((np.square(star['x']) +
                    np.square(star['y']) +
                    np.square(star['z'])) < gal.meta.rgal**2)[0]# in kpc unit

    gal.star = star[ind]

    if gal.debug:
        print('[galaxy.Galaxy.mk_gal] mima vx 1', min(gal.star['vx']), max(gal.star['vx']))


    gal.meta.nstar = len(ind)
    gal.meta.mstar = sum(gal.star['m'])

    if gal.meta.mstar < mstar_min:
        print("Not enough stars: {:.2e} Msun".format(gal.meta.mstar))
        print("{} Aborting... \n".format(len(gal.star['m'])))
        gal.meta.star = False
        return False

    gal.meta.Rgal_to_reff = gal.meta.rgal / gal.meta.reff
    # should not surpass rr_tmp, over where another galaxy might be.

    # Test if the outer annulus has significant amount of stars
    # -> it shouldn't.
#        if verbose: print("Reff: ", reff_tmp)

    # extract galaxy (particles and gas)
    if star is not None:
        nstar_tot = len(star['x'][ind])
        if verbose: print("nstar tot:", nstar_tot)
        if verbose: print("Store stellar particle")

        if 'time' in gal.star.dtype.names and unit_conversion == "code":
            import utils.cosmology
            gal.star['time'] = utils.cosmology.time2gyr(gal.star['time'],
                                         z_now = gal.info.zred,
                                         info=gal.info)

    # VERY arbitrary..
#        gal.Rgal_to_reff = 4 #rgal_tmp / reff_tmp
    rgal_tmp = gal.meta.Rgal_to_reff *gal.meta.reff

    #print(".........", gal.star['m'][100:120], gal.mstar)
    import utils.sampling as smp
    gal.region = smp.set_region(xc=gal.meta.xc, yc=gal.meta.yc, zc=gal.meta.zc, radius = gal.meta.rgal)

    # Now, get cov
#        gal.get_cov(center_only=True)
    #if gal.star is not None:
        #gal._convert_unit_meta(unit_conversion)
        #gal._convert_unit("star", unit_conversion)

    if gal.debug:
        print('[galaxy.Galaxy.mk_gal] meta.v[x,y,z]c', gal.meta.vxc, gal.meta.vyc, gal.meta.vzc)
        print('[galaxy.Galaxy.mk_gal] mima vx 2', min(gal.star['vx']), max(gal.star['vx']))

    if dm is not None:
        if member == "Reff":
            idm = np.where( np.square(dm["x"] - gal.meta.xc) +
                            np.square(dm["y"] - gal.meta.yc) +
                            np.square(dm["z"] - gal.meta.zc) <= np.square(rgal_tmp))[0]
        elif member == "v200":
        # Although the velocity is redefined later,
        # particle membership is fixed at this point.
            idm = np.where( np.square(dm["vx"] - gal.meta.vxc / gal.info.kms)+
                            np.square(dm["vy"] - gal.meta.vyc / gal.info.kms)+
                            np.square(dm["vz"] - gal.meta.vzc / gal.info.kms) <= np.square(200**2))[0]
        gal._convert_unit("dm", unit_conversion)

    if cell is not None:
        if verbose: print("Cell is NOT none")
        icell = np.where(np.square(cell["x"] - xc) +
                         np.square(cell["y"] - yc) +
                         np.square(cell["z"] - zc) <= np.square(rgal_tmp))[0]

        gal._add_cell(cell, icell)
        gal._convert_unit("cell", unit_conversion)
        gal.cal_mgas()

        gal.cell = np.recarray(ncell_tot, dtype=dtype_cell)
        gal.cell['x'] = (cell['x'][icell] - xc) * gal.info.pboxsize * 1000
        gal.cell['y'] = (cell['y'][icell] - yc) * gal.info.pboxsize * 1000
        gal.cell['z'] = (cell['z'][icell] - zc) * gal.info.pboxsize * 1000
        gal.cell['dx'] = cell['dx'][icell] * gal.info.pboxsize * 1000
        gal.cell['rho'] = cell['var0'][icell]
        gal.cell['vx'] = cell['var1'][icell] * gal.info.kms - gal.meta.vxc
        gal.cell['vy'] = cell['var2'][icell] * gal.info.kms - gal.meta.vyc
        gal.cell['vz'] = cell['var3'][icell] * gal.info.kms - gal.meta.vzc
        gal.cell['temp'] = cell['var4'][icell]
        gal.cell['metal'] = cell['var5'][icell]


    # Some more sophistications.
    """
    print("Rgal = 4 * Reff = ", rgal_tmp * gal.info.pboxsize * 1000)

        # Save sink particle as a BH, not cloud particles.

    """
    return True


def radial_profile_cut(gal, xx, yy, mm, vx, vy, vz,
                       den_lim=1e6, den_lim2=5e6,
                       mag_lim=25, nbins=100, rmax=20, dr=0.5,
                       debug=False):
    # 2D photometry. (if rotated towards +y, then use x and z)
    # now assuming +z alignment.
    rr = np.sqrt(np.square(xx) + np.square(yy))# in kpc unit
    if debug:
        print("min(rr) {}\n max(rr){}\n min(xx){}\n max(xx){}".format(
                               min(rr), max(rr), min(xx), max(xx)))

    # Account for weights.
    i_sort = np.argsort(rr)
    r_sorted = rr[i_sort]
    m_sorted = mm[i_sort]

    rmax = min([np.max(rr), rmax])
    nbins = int(rmax/dr)

    if nbins < 3:
        print("Too small size \n # of stars:", len(rr))
        return False

    frequency, bins = np.histogram(r_sorted, bins = nbins, range=[0, rmax])
    bin_centers = bins[:-1] + 0.5 * dr # remove the rightmost boundary.

    m_radial = np.zeros(nbins)
    ibins = np.concatenate((np.zeros(1,dtype=int), np.cumsum(frequency)))

    i_r_cut1 = nbins -1 # Maximum value
    # on rare occasions, a galaxy's stellar surface density
    # never crosses the density limit. Then i_r_cut1 = last index.
    for i in range(nbins):
        m_radial[i] = np.sum(m_sorted[ibins[i]:ibins[i+1]])
        # Check stellar surface density
        sig_at_r = m_radial[i]/(2 * np.pi * bin_centers[i] * dr)
        if debug:
            print(sig_at_r, den_lim)
        if sig_at_r < den_lim:
            i_r_cut1 = i-1
            break
    #i_r_cut2= np.argmax(m_radial/(2 * np.pi * bin_centers * dr) < den_lim2)
    # If for some reason central region is less dense,
    # profile can end at the first index.
    # Instead coming from backward, search for the point the opposite condition satisfied.
    if debug:
        print(rmax, nbins)
        print("frequency", frequency)
        print("bins", bins)
        print("ibins", ibins)
        print("bin centers", bin_centers)
        print("m_radial", m_radial)

    den_radial_inverse = m_radial[::-1]/(2 * np.pi * bin_centers[::-1] * dr)
    if debug: print("den_radial_inverse", den_radial_inverse)
    if max(den_radial_inverse) < 2 * den_lim2:
        np.set_printoptions(precision=3)
        print("radial density :",den_radial_inverse)
        return False
    i_r_cut2=len(m_radial) - np.argmax(den_radial_inverse > den_lim2) -1
    if debug:
        print("[galaxy.Galaxy.radial_profile_cut] m_radial \n", m_radial)
        print("[galaxy.Galaxy.radial_profile_cut] den_radial_inverse \n", den_radial_inverse)
        print("[galaxy.Galaxy.radial_profile_cut] i_r_cut2", i_r_cut2)

    mtot2 = sum(m_radial[:i_r_cut2])
    mtot1 = sum(m_radial[:i_r_cut1])
    i_reff2 = np.argmax(np.cumsum(m_sorted) > (0.5*mtot2))
    i_reff1 = np.argmax(np.cumsum(m_sorted) > (0.5*mtot1))
    gal.meta.reff2 = r_sorted[i_reff2]
    gal.meta.reff  = r_sorted[i_reff1]
    #print(bin_centers, i_r_cut2, m_radial)
    gal.meta.rgal2 = max([bin_centers[i_r_cut2],4*gal.meta.reff2])
    gal.meta.rgal  = max([bin_centers[i_r_cut1],4*gal.meta.reff])#bin_centers[i_r_cut1]

#       velocity center
#       It is not wrong for BCGs to have very large Reff(~50kpc).
#       But referring the average velocity of stellar particles inside 50kpc
#       as the system velocity is WRONG.
#       If 1Reff is huge, try smaller aperture when measuring the system velocity.
#
    if debug:
        print("[galaxy.Galaxy.radial_profile_cut] mtot, mtot2", mtot1, mtot2)

    i_close = i_sort[:np.argmax(np.cumsum(m_sorted) > (0.2*mtot2))] # 20% closest particles


    gal.meta.vxc = np.average(vx[i_close])
    gal.meta.vyc = np.average(vy[i_close])
    gal.meta.vzc = np.average(vz[i_close])

    return True

In [5]:
def get_normal_vec(stars):
    #vec_rot = np.cross(stars["pos"],stars["vel"])
    vec_rot = np.cross(np.vstack((stars["x"],stars["y"],stars["z"])),
                       np.vstack((stars["vx"],stars["vy"],stars["vz"])))
    Ln = vec_rot.sum(axis=0) 
    return Ln / np.linalg.norm(Ln)

In [7]:
iif = load.info.Info(nout=782)

In [6]:
nout=782
Mcut = 1e10

cal_lambda_params = dict(npix_per_reff=5,
                        rscale=3.0,
                        method='ellip',
                        n_pseudo=1,
                        verbose=False,
                        voronoi=None,#voronoi_dict,
                        mge_interpol = True)
mk_gal_params = dict()

# Load data
import tree


s = load.sim.Sim(nout=nout)
# Check for tree file before reading.
# Or, the fortran routine crashes causing the kernel crash without any error message. 

[sim._hilbert_cpulist] No AMR instance,
[sim._hilbert_cpulist] Loading one...
An AMR instance is created
 Sim
An AMR instance is created
 Sim
Simulation set up.


In [ ]:
all_gcats=[]
for nout in nnza["nout"]:
    all_gcats.append(tree.halomodule.Halo(nout=nout, is_gal=True))

[Halo.load_info] loading info
[Halo.load_info] nout = 782, base =./


In [30]:

gcat = tree.halomodule.Halo(nout=nout, is_gal=True)


[Halo.load_info] loading info
[Halo.load_info] nout = 782, base =./


In [7]:
from galaxymodule import mk_gal_params as mgp

mgp.HAGN["verbose"] = False

In [8]:
tt = tree.tmtree.Tree(is_gal=True)

In [15]:
tnow = tt.tree[tt.tree["nstep"]==max(tt.tree["nstep"])]
large_last = tnow[(tnow["m"] > 1) * (tnow["m"] < 2)]
print(large_last["idx"])


[7864648 7864649 7864653 ..., 7985705 7986018 7986759]


In [16]:
final_idxs = large_last["idx"][large_last["id"] % 10 == 5]
final_ids = large_last["id"][large_last["id"] % 10 == 5]

In [67]:
len(final_idxs)

183

In [72]:
tt.aexps

array([ 0.145,  0.157,  0.169,  0.179,  0.19 ,  0.2  ,  0.211,  0.221,
        0.231,  0.24 ,  0.25 ,  0.258,  0.268,  0.278,  0.284,  0.286,
        0.295,  0.303,  0.311,  0.32 ,  0.337,  0.353,  0.37 ,  0.386,
        0.401,  0.417,  0.432,  0.448,  0.463,  0.478,  0.493,  0.508,
        0.523,  0.538,  0.553,  0.568,  0.583,  0.598,  0.613,  0.628,
        0.643,  0.658,  0.673,  0.689,  0.704,  0.719,  0.735,  0.75 ,
        0.766,  0.782,  0.798,  0.814,  0.83 ,  0.846,  0.863,  0.879,
        0.896,  0.913,  0.93 ,  0.947,  0.965,  0.982], dtype=float32)

In [74]:
#no, ns, zr, ae = 
nnza = np.genfromtxt("./nout_nstep_zred_aexp.txt",
                     dtype=[("nout", int),
                            ("nstep", int),
                            ("zred", float),
                            ("aexp", float)])

In [80]:
for aexp in tt.aexps:
    print(nnza["nout"][np.abs(aexp - nnza["aexp"]).argmin()])

43
57
70
83
90
97
102
107
112
118
125
131
139
147
152
154
161
168
175
183
197
213
230
248
266
278
290
302
315
327
343
358
374
389
406
422
439
454
470
486
503
519
535
552
570
587
604
621
638
654
670
680
691
702
713
723
733
742
752
761
772
782


In [64]:
merger_dist_all=[]
for fid,fidx in zip(final_ids, final_idxs):
    maintree, idx_prgs_alltime = tt.extract_direct_full_tree(fidx)
    all_direct_prgs = tt.get_all_trees(idx_prgs_alltime)

    
    all_step=[]
    for this_step in all_direct_prgs[1:2]:
        step_now = 782
        gg = load.rd_GM.Gal(nout=step_now,
                            catalog=np.copy(gcat.data[fid-1]),
                            info=s.info)
        gg.debug=False
        #gg.mk_gal(**mgp.HAGN)
        mk_gal(gg,**mgp.HAGN)
        gg.cal_norm_vec()
        # Make sure that the catalog is not being modified.
        
        all_gal = []
        for this_gal in this_step:
            this_gal_relang=[]
            #this_gal_relang.append(0)
            if this_gal is None:
                continue
            for gal in this_gal[0:1]: 
                #only at the last moment
                #gal = this_gal[0]
                print(gal["nstep"])
                rel_pos = (gg.header["xg"] - gal["xp"])*1e3 # in kpc
                rel_vel = gg.header["vg"] - gal["vp"] 
                jx,jy,jz = np.cross(rel_pos, rel_vel)
                j_orbital=(jx,jy,jz)/np.sqrt(jx**2 + jy**2 + jz**2)
                # rotation axis
                relang = 180 / np.pi * np.arccos(np.dot(gg.meta.nvec, j_orbital))
                this_gal_relang.append(relang)
            all_gal.append(this_gal_relang)
        #print("---")
        all_step.append(all_gal)
        
    merger_dist_all.append(all_step)

0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
56-th step
57-th step
58-th step
59-th step
60-th step
61-th step
62-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0000045
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0000045
No CELL data loaded
gal.debug False
61
61
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step

/home/hoseung/Work/pyclusterevol/load/rd_GM.py:112: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  assert(not(idgal == None and catalog == None)), ("either idgal or a catalog"



9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
56-th step
57-th step
58-th step
59-th step
60-th step
61-th step
62-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0000095
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0000095
No CELL data loaded
gal.debug False
61
61
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step


3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
56-th step
57-th step
58-th step
59-th step
60-th step
61-th step
62-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0000385
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0000385
No CELL data loaded
gal.debug False
61
61
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th s

7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0000635
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0000635
No CELL data loaded
gal.debug False
61
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th 

...

17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0006245
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0006245
No CELL data loaded
gal.debug False
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th 

9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
56-th step
57-th step
58-th step
59-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0007085
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0007085
No CELL data loaded
gal.debug False
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th s

19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
56-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0008705
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0008705
No CELL data loaded
gal.debug False
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th 

4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
56-th step
57-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0014055
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0014055
No CELL data loaded
gal.debug False
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
1

4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0115935
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0115935
No CELL data loaded
gal.debug False
61
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th ste

In [41]:
fig, ax = plt.subplots()
for all_step, idx in zip(merger_dist_all[2:3],final_idxs[2:3]):
    for i, this_step in enumerate(all_step):
        for all_gal in this_step:
            ax.scatter(t_lookback[i],all_gal, c=cm.hot(i*4))
            #for this_gal in all_gal:
                #print(this_gal)
            #print("--")
    ax.set_ylim([0,180])
    ax.set_title("Merger angle")
    ax.set_xlabel("lookback time")
    ax.set_ylabel("degree")
    plt.show()#savefig("Merger_angle_dist_"+str(idx).zfill(8)+".png")
    #plt.cla()

In [32]:
t_lookback = tt.age_univs[-1] - np.array(tt.age_univs[::-1])

In [45]:
from matplotlib import cm
fig, ax = plt.subplots()

for all_step, idx in zip(merger_dist_all,final_idxs):
    for i, this_step in enumerate(all_step):
        for all_gal in this_step:
            ax.scatter(t_lookback[i],all_gal, c=cm.hot(i*4))
            #for this_gal in all_gal:
                #print(this_gal)
            #print("--")
    ax.set_ylim([0,180])
    ax.set_title("Merger angle")
    ax.set_xlabel("lookback time")
    ax.set_ylabel("degree")
    plt.savefig("Merger_angle_dist_"+str(idx).zfill(8)+".png")
    plt.cla()

In [65]:
l = [item for sublist in merger_dist_all for item in sublist]
ll = [item for sublist in l for item in sublist]
lll = np.array(ll).ravel()

In [69]:
fig, ax = plt.subplots()
ax.hist(lll)
ax.set_title("relaive angle distribution (183 galaxies, 1~2e11M*, z=0)")
ax.set_xlabel("degree")
ax.set_ylabel("#")
plt.show()


# B/T 

In [70]:
gg.cal_b2t()

TypeError: 'NoneType' object is not subscriptable

In [18]:
all_step=[]
for this_step in all_data[1:]:
    all_gal = []
    for this_gal in this_step:
        this_gal_relang=[]
        #this_gal_relang.append(0)
        if this_gal is None:
            continue
        for gal in this_gal[0:1]: #only at the last moment
            #gal = this_gal[0]
            #print(gal)
            rel_pos = (gg.header["xg"] - gal["xp"])*1e3 # in kpc
            rel_vel = gg.header["vg"] - gal["vp"] 
            jx,jy,jz = np.cross(rel_pos, rel_vel)
            j_orbital=(jx,jy,jz)/np.sqrt(jx**2 + jy**2 + jz**2)
            # rotation axis
            relang = 180 / np.pi * np.arccos(np.dot(gg.meta.nvec, j_orbital))
            this_gal_relang.append(relang)
        all_gal.append(this_gal_relang)
    #print("---")
    all_step.append(all_gal)